In [ ]:
from pathlib import Path
import pandas as pd

path = Path("~/.finances").expanduser() / "recurring_transactions" / "20240901_rec_trans.csv"
rec_trans = pd.read_csv(path)
rec_trans["amount"] = rec_trans["amount"].replace(r'[\$, ]', '', regex=True)
rec_trans["amount"] = pd.to_numeric(rec_trans["amount"], errors="coerce")
rec_trans.head()

In [ ]:
"""
def project_recurring_transactions(
    rec_trans: df,
    projection length: int - ex: 3 months into future
) -> df:
    Create a new, empty df (cash_flow) that will be the function output
    frequency_map = {dict mapping frequency column (weekly, monthly, etc) with how to update current_date}
    start_date = datetime.now().date()
    projection_end_date = 3 months after start_date
    For each row in rec_trans df:
        if start_date is later than end_date column value:
            print warning message to consider updating rec_trans csv
            continue
        current_date = start_date
        prorate_statement_closing = True
        while current_date <= projection_end_date:
            if precise_on_date == False and prorate_statement_closing == True:
                transaction_amount = prorated amount
                prorate_statement_closing = False # Only first instance of that transaction type gets prorated
            else:
                transaction_amount = full amount
            add a new row to cash_flow with transaction_amount, date (equal to current_date), transaction name, category, and charge_to
            increment current_date based on frequency/frequency_map
    return cash_flow


def project_supplemental_transactions(
    rec_trans_projection: df,
    maybe also start date and/or projection length to keep user from entering out-of-range transactions?
) -> df:
    Gui interface with user to get extra transactions (placeholder text with rec_trans_projection for becca OT rows)
    return a new df with a row for each extra/supplemental transaction (df has same cols as rec_trans_projection)


rec_trans_projection = project_recurring_transactions(...)
supp_trans_projection = project_supplemental_transactions(...)
merge the 2 dataframes
sort the newly merged df
add a current balances row and current bank, cc, statement balances columns (and also possibly banks-cc statement and banks-cc total cols)
add rows at end of all 11th and 13th transactions for statement paid and statement closing
execute all of the calculations row by row to fill in all of the new columns for banks and cc balances
"""

In [ ]:
from pathlib import Path
import pandas as pd

path = Path("~/.finances").expanduser() / "current_balances" / "20240901_curr_bals.csv"
curr_bals = pd.read_csv(path)
cols = ["statement_balance", "current_balance"]
for col in cols:
    curr_bals[col] = curr_bals[col].replace(r'[\$, ]', '', regex=True)
    curr_bals[col] = pd.to_numeric(curr_bals[col], errors="coerce")
curr_bals.head()

In [34]:
curr_bank_bal = curr_bals.loc[curr_bals.type == "bank", "current_balance"].sum()
curr_bank_bal

11029.0

In [ ]:
grouped_curr_balances = curr_bals.groupby(["type"]).agg(
    statement_balance_sum=("statement_balance", "sum"),
    current_balance_sum=("current_balance", "sum")
).reset_index()
grouped_curr_balances.head()

In [ ]:
curr_bank_bal = grouped_curr_balances.loc[grouped_curr_balances.type == "bank", "current_balance_sum"].values[0]
curr_ccstmt_bal = grouped_curr_balances.loc[grouped_curr_balances.type == "cc", "statement_balance_sum"].values[0]
curr_cctotal_bal = grouped_curr_balances.loc[grouped_curr_balances.type == "cc", "current_balance_sum"].values[0]
curr_inv_bal = grouped_curr_balances.loc[grouped_curr_balances.type == "investment", "current_balance_sum"].values[0]

print(curr_bank_bal, curr_ccstmt_bal, curr_cctotal_bal, curr_inv_bal)